In [ ]:
import hoss
import os
import time
from pathlib import Path
import shutil

## Connect to local server
This notebook demonstrates basic operations using a single Hoss server.
For these demo notebooks, it's assumed you have the `admin` role and are running the server locally
on localhost. If using a different server, be sure to change the endpoint in the `.connect()` call.

We start by connecting the the "local" server. 

In [ ]:
server_local = hoss.connect('http://localhost')

In [ ]:
print("Existing Namespaces:")
print(server_local.list_namespaces())

## Create a dataset
First load the default namespace and then create a dataset inside the namespace

In [ ]:
ns = server_local.get_namespace('default')

In [ ]:
ds = ns.create_dataset("download-test", "A dataset for a simple example")
ds.display()

## Download file function

The client library provides everything you need to deal with downloading data

We can create a simple function that will download a ref to a specified root directory.

In [ ]:
def download_ref(root_dir, ref):
    if ref.key[-1] == "/":
        return
    
    dest_file = Path(root_dir, ref.key)
    dest_dir = dest_file.parents[0]
    dest_dir.mkdir(parents=True, exist_ok=True)
    
    dest_file = Path(root_dir, ref.key)
    with open(dest_file, 'wb') as df:
        with ref.open('rb') as fh:
            df.write(fh.read())

## Download files in a dataset

Using `rglob` or `iterdir` we can easily download all files or files in a directory.

In [ ]:
# write a bit more data
f1 = ds / "foo0.txt"
f1.write_text("foo0")
f1 = ds / "folder1" / "foo1.txt"
f1.write_text("foo1")
f1 = ds / "folder1" / "foo2.txt"
f1.write_text("foo2")
f1 = ds / "folder1" / "foo3.txt"
f1.write_text("foo3")
f1 = ds / "folder1" / "bar0.txt"
f1.write_text("bar0")

In [ ]:
for f in (ds).rglob("*"):
    print(f)

In [ ]:
root_dir = Path(".", 'test-dir')
for f in (ds).rglob("*"):
    download_ref(root_dir, f)

In [ ]:
# Clean up local test dir
shutil.rmtree(root_dir)

In [ ]:
root_dir = Path(".", 'test-dir')
for f in (ds / "folder1").iterdir():
    download_ref(root_dir, f)

In [ ]:
# Clean up local test dir
shutil.rmtree(root_dir)

## Use search to download data
You can use the same method with a `search_ref` call to download files that meet some criteria

Available arguments to `search` method:
- metadata: dictionary of key-value pairs that must match
- namespace: name of namespace to filter results
- dataset: name of dataset to filter results (namespace must be set along with dataset to be valid)
- modified_before: datetime string format `2006-01-02T15:04:05.000Z` to filter results
- modified_after: datetime string format `2006-01-02T15:04:05.000Z` to filter results
- limit: number of items to return per page
- offset: starting point in the index for returned items

In [ ]:
f2= ds / "with-meta"/ "test2.txt"
f2.write_text("my data file", metadata={'foo': 'bar', 'fizz': 'buzz'})
f3 = ds / "with-meta"/ "test3.txt"
f3.write_text("my data file", metadata={'foo': 'bar', 'fizz': 'other'})
f4 = ds / "with-meta"/ "test4.txt"
f4.write_text("my data file", metadata={'foo': 'bar'})
time.sleep(2)

In [ ]:
server_local.search({'foo': 'bar'})

In [ ]:
results = server_local.search_refs({'foo': 'bar'})
root_dir = Path(".", 'test-dir')
for f in results:
    download_ref(root_dir, f)

In [ ]:
# Clean up local test dir
shutil.rmtree(root_dir)

## Clean up this example

Run these cells to remove the resources created during the test

In [ ]:
ns.delete_dataset("download-test")